In [34]:
from keras.layers import Input, Dense
from keras.models import Model

input_all = Input(shape=(233,), name="input")

x = Dense(512,activation='relu')(input_all)

x = Dense(256,activation= 'relu')(x)

x = Dense(128)(x)

out = Dense(4,activation='sigmoid')(x)

model = Model(inputs=[input_all], outputs=out)
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 233)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 512)               119808    
_________________________________________________________________
dense_63 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_64 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_65 (Dense)             (None, 4)                 516       
Total params: 284,548
Trainable params: 284,548
Non-trainable params: 0
_________________________________________________________________


In [35]:
from emulator import CustomEmulator
import numpy as np

emul = CustomEmulator(500,10)
num_hands = 5000
total_reward = 0
y = .99
e = 0.1

for i in range(num_hands):
    reward_sb = 0
    reward_bb = 0
    #Starts with dealer
    is_sb_turn = True
    
    sb_has_played = False
    bb_has_played = False
  
    emul.new_hand()
    #while hand is still at play
    while(emul.is_round_finished() == False):
           
        emul.make_features()
    
        if(is_sb_turn == True):
            #save old state to update it asap with new rew
            if(sb_has_played == True):
                old_state = sb_features
                targetQ = allQ_sb
                oldAction = action_sb
                
                ##tochange
            sb_features = np.concatenate([emul.players_cards[0], np.concatenate(
    emul.cards_feature), np.concatenate(emul.actions_feature),np.array([1])]).reshape((1,233))
            
            #run model to choose action
            allQ_sb = model.predict(sb_features)
            action_sb = np.argmax(allQ_sb)
            #print(allQ_sb)
            
            #randomly discover new line
            if(np.random.rand(1) < e):
                action_sb = np.random.randint(0,4)
            
            emul.play_action(action_sb)
                
            
            if(sb_has_played == True):
                reward_sb += y*np.max(allQ_sb)
                targetQ[0,action_sb] = reward_sb
                model.fit(old_state,targetQ,verbose=0)
                
            sb_has_played = True
            is_sb_turn = False
        else:
            #save old state to update it asap with new rew
            if(bb_has_played == True):
                old_state = bb_features
                targetQ = allQ_bb
                oldAction = action_bb
                
                ##tochange
            bb_features = np.concatenate([emul.players_cards[0], np.concatenate(
    emul.cards_feature), np.concatenate(emul.actions_feature),np.array([1])]).reshape((1,233))
            
            #run model to choose action
            allQ_bb = model.predict(bb_features)
            action_bb = np.argmax(allQ_bb)
            
            #randomly discover new line
            if(np.random.rand(1) < e):
                action_bb = np.random.randint(0,4)
                
            emul.play_action(action_bb)
                
        
            
            if(bb_has_played == True):
                reward_bb += y*np.max(allQ_bb)
                targetQ[0,action_bb] = reward_bb
                model.fit(old_state,targetQ,verbose=0)


            bb_has_played = True
            is_sb_turn = True
     
    r  = emul.get_sb_reward()/500
    total_reward +=r
    
    if(i%250 == 0):
        print('round',i)

        print('total won (SB):',total_reward)
        print(allQ_sb)
        #print(emul.events)

        if(bb_has_played):
            print(allQ_bb)
    
    #update sb with ACTUAL value
    allQ_sb[0,action_sb] = r
    model.fit(sb_features,allQ_sb,verbose=0)
    
    if(bb_has_played):   
        allQ_bb[0,action_bb] = -r

        model.fit(bb_features,allQ_bb,verbose=0)
  
            

round 0
total won (SB): 0.1
[[ 0.51124781  0.47716764  0.49000287  0.51293433]]
[[ 0.51639903  0.47701684  0.49760175  0.65430093]]
round 250
total won (SB): -1.5600000000000005
[[ 1.          1.          1.          0.99975175]]
round 500
total won (SB): -5.359999999999978
[[ 1.          1.          1.          0.99999857]]
round 750
total won (SB): -7.4399999999998965
[[ 1.          0.99999976  1.          0.99987316]]
round 1000
total won (SB): -10.679999999999854
[[ 1.         0.9998703  1.         0.9999994]]
round 1250
total won (SB): -13.239999999999853
[[ 1.          0.99946219  1.          0.99999881]]


KeyboardInterrupt: 